add description

In [31]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain.document_loaders import PyPDFLoader
from langchain_core.documents import Document

In [32]:
loader = PyPDFLoader('../data/agentic-ai-system.pdf')
corpus = loader.load()

In [33]:
len(corpus)

288

In [34]:
corpus[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': '../data/agentic-ai-system.pdf', 'total_pages': 288, 'page': 0, 'page_label': 'i'}, page_content='')

Document(metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': '../data/agentic-ai-system.pdf', 'total_pages': 288, 'page': 0, 'page_label': 'i'}, page_content='')

In [35]:
corpus = corpus[12:]

In [36]:
len(corpus)

276

In [37]:
corpus[111]

Document(metadata={'producer': 'Adobe PDF Library 17.0; modified using iText® 5.5.13 ©2000-2018 iText Group NV (AGPL-version)', 'creator': 'Adobe InDesign 18.5 (Windows)(Foxit Advanced PDF Editor)', 'creationdate': '2025-04-01T12:11:31+05:30', 'icnappname': 'Foxit Advanced PDF Editor', 'icnappplatform': 'Windows', 'icnappversion': '3.05', 'moddate': 'D:20250424033741', 'trapped': '/False', 'source': '../data/agentic-ai-system.pdf', 'total_pages': 288, 'page': 123, 'page_label': '99'}, page_content='Use cases and examples 99\nserve user needs. In addition to content analysis, reflective chatbots can also assess the effectiveness \nof their communication styles and language usage. By analyzing user feedback and reactions, they can \ndetermine which tones, wordings, or levels of formality resonate better with different user groups or \ncontexts. This insight can then inform the chatbot’s ability to adapt its communication style dynamically, \nfostering more natural and personalized intera

In [38]:
corpus[111].page_content

'Use cases and examples 99\nserve user needs. In addition to content analysis, reflective chatbots can also assess the effectiveness \nof their communication styles and language usage. By analyzing user feedback and reactions, they can \ndetermine which tones, wordings, or levels of formality resonate better with different user groups or \ncontexts. This insight can then inform the chatbot’s ability to adapt its communication style dynamically, \nfostering more natural and personalized interactions.\nMoreover, self-assessment can help chatbots identify knowledge gaps or areas where their understanding \nis limited. By recognizing instances where they struggle to provide satisfactory responses, chatbots can \nproactively seek out additional information or consult with human experts to expand their knowledge \nbase and improve their ability to handle a wider range of queries effectively.\nSoftware companies such as Zendesk and Drift use AI-powered chatbots that learn from conversations. 

In [39]:
corpus = [page.page_content for page in corpus]

In [40]:
corpus[22]

'Fundamentals of Generative AI 10\n• Domain-specific LLMs: As mentioned earlier, while LLMs are often trained on a large set \nof open, generally available, web data, they may not perform very well in domain-specific \nenvironments. While instruction-tuned LLMs are an avenue to make these general-purpose \nmodels work for domain-specific tasks, researchers have innovated on training domain-specific \nLLMs from the ground up. This means that these models were pre-trained on highly specialized, \ndomain knowledge. For example, the BioBERT model is trained on hyper-specialized medical \ndata, whereas LegalBERT is trained on legal documents. These models are useful for specialized \ntasks in specific fields such as medicine, law, and finance.\nLLM-powered AI agents\nEach of the previously discussed LLM types has its own strengths and weaknesses and is well-suited \nfor different applications. However, our focus in this book would be to dive deeper into LLM agent \ncapabilities and explore 

In [41]:
type(corpus)

list

In [42]:
corpus[123]

'Understanding the concept of tool use in agents 111\nIt is worth noting that not all LLMs are capable or efficient (or rather accurate) in tool/function calling. \nWhile larger models are more capable of tool calling, some larger models (such as OpenAI’s GPT-4 \nand -4o, Anthropic’s Claude Sonnet, Haiku, Opus, and Meta’s Llama 3 models) are explicitly trained \nfor tool calling behavior. While other models are not explicitly trained on tool calling, they may still \nbe able to achieve similar functionality with aggressive prompt engineering, but with varying degrees \nof success.\nDefining tools for agents\nTools are defined with clear descriptions, typically using docstrings or a JSON schema, to communicate \ntheir purpose, required inputs, and expected outputs to the agent. There are two main approaches to \ndefining tools, depending on whether you’re using a framework or working directly with LLM APIs.\nFramework approach – using docstrings\nIn frameworks such as CrewAI or LangGrap

In [44]:
document = [
            Document(page_content=text, metadata={'page': idx+1}) 
            for idx, text in enumerate(corpus)
            ]

In [47]:
document[222]

Document(metadata={'page': 223}, page_content='Managing Safety and Ethical Considerations 210\nFor example, an agentic AI system in healthcare might not only have access to historical medical \nrecords for training but also actively manage patient scheduling, treatment plans, and medical device \nsettings. If such a system mishandles private information, it could autonomously share sensitive medical \ndetails with unauthorized parties, schedule appointments that reveal confidential conditions, or make \ntreatment decisions that inadvertently expose protected health information.\nIn the travel industry, privacy violations could occur when agentic systems go beyond simple data \nexposure to actively making privacy-compromising decisions. An autonomous travel assistant might \nnot just leak travel itineraries but could also make bookings that reveal sensitive personal information, \nautomatically share location data with third parties, or create patterns of behavior that expose confidenti

In [48]:
print(type(document[0]))

<class 'langchain_core.documents.base.Document'>
